## 1、tensorflow版本查看

In [1]:
import tensorflow as tf

print('Tensorflow Version:{}'.format(tf.__version__))
print(tf.config.list_physical_devices())

2024-05-04 21:29:05.936344: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow Version:2.16.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## 2、MNIST数据集下载与预处理

In [2]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_images,test_images = tf.cast(train_images/255.0,tf.float32),tf.cast(test_images/255.0,tf.float32) # 归一化

### tf.data.Dataset制作训练数据集

In [3]:
ds_train_image = tf.data.Dataset.from_tensor_slices(train_images)
ds_train_label = tf.data.Dataset.from_tensor_slices(train_labels)
ds_train = tf.data.Dataset.zip((ds_train_image,ds_train_label))
ds_train = ds_train.shuffle(10000).repeat().batch(64) # 乱序，无限次重复，每次取64张图片

print(ds_train_image)
print(ds_train_label)
print(ds_train)

<_TensorSliceDataset element_spec=TensorSpec(shape=(28, 28), dtype=tf.float32, name=None)>
<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.uint8, name=None)>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


### tf.data.Dataset制作测试数据集

In [4]:
ds_test = tf.data.Dataset.from_tensor_slices((test_images,test_labels))
ds_test = ds_test.repeat().batch(64)

print(ds_test)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


## 3、搭建MLP模型

In [5]:
from keras import Sequential
from keras.layers import Flatten,Dense
from keras import Input

model = Sequential()
model.add(Input(shape=(28,28)))
model.add(Flatten())
model.add(Dense(units=256,kernel_initializer='normal',activation='relu'))
model.add(Dense(units=10,kernel_initializer='normal',activation='softmax'))
# model.summary()

## 4、模型训练

### 调用model.compile()函数对训练模型进行设置

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 调用model.fit()配置训练参数，开始训练，并保存训练结果。

In [7]:
steps_per_epochs = train_images.shape[0]//64  # 937

H = model.fit(ds_train,  # 训练数据集
              steps_per_epoch=steps_per_epochs,  # 每个epoch训练步数
              validation_data=ds_test,  #验证数据集
              validation_steps=10000//64,
              epochs=10,
              verbose=1)

Epoch 1/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8639 - loss: 0.4951 - val_accuracy: 0.9561 - val_loss: 0.1449
Epoch 2/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9619 - loss: 0.1308 - val_accuracy: 0.9692 - val_loss: 0.1016
Epoch 3/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9751 - loss: 0.0824 - val_accuracy: 0.9738 - val_loss: 0.0850
Epoch 4/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9831 - loss: 0.0568 - val_accuracy: 0.9762 - val_loss: 0.0774
Epoch 5/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9866 - loss: 0.0450 - val_accuracy: 0.9769 - val_loss: 0.0703
Epoch 6/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9903 - loss: 0.0335 - val_accuracy: 0.9778 - val_loss: 0.0709
Epoch 7/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9926 - loss: 0.0257 - val_accuracy: 0.9798 - val_loss: 0.0716
Epoch 8/10
937/937 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9945 - loss: 0.0202 - val_accuracy: 0.